# **Google drive for local storage**


_NB: All comments are written to facilitate smooth evaluation of the model, that the **Current User** may be less fatigued and see beauty in the good work._

Uncomment text under **PREVIEW OUTPUT** to further scrutinize.

In [1]:
# This cell will prompt an external url to accept permissions for Colab to access Google Drive

from google.colab import drive
drive.mount("/gdrive")

%ls

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
corpus.txt    NWP-USE/        puntuated_.txt  tokenizer1.pkl    vocabulary.npy
nextword1.h5  pred_model4.h5  sample_data/    tokenizer.pickle


# **Import ***

In [2]:
# Getting all required libraries

import os
import re
import gdown
import numpy
import string
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from absl import logging
import tensorflow_hub as hub
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.models import Sequential
import tensorflow.keras.backend as K
from keras.layers.recurrent import LSTM
from keras.layers import Dense, Activation
from keras.callbacks import LambdaCallback
from keras.utils.data_utils import get_file
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split

## **Data preparation - _Generating Corpus_**

In [3]:
# Download data from Google drive

'''

ORIGINAL DATASET URL:
    https://raw.githubusercontent.com/maxim5/stanford-tensorflow-tutorials/master/data/arxiv_abstracts.txt

'''

url = ' https://drive.google.com/uc?id=1YTBR7FiXssaKXHhOZbUbwoWw6jzQxxKW'
output = 'corpus.txt'
gdown.download(url, output, quiet=False)

# sentence_length = 40

# Read local file from directory
with open('corpus.txt') as subject:
  cache = subject.readlines()
translator = str.maketrans('', '', string.punctuation) # Remove punctuation
lines = [doc.lower().translate(translator) for doc in cache] # Switch to lower case

Downloading...
From:  https://drive.google.com/uc?id=1YTBR7FiXssaKXHhOZbUbwoWw6jzQxxKW
To: /content/corpus.txt
7.55MB [00:00, 171MB/s]


In [4]:
# PREVIEW OUTPUT ::

# print(lines[0][:100])
# len(lines)

In [5]:
# Generate an list of single/independent words

vocabulary = list(set(' '.join(lines).replace('\n','').split(' ')))
primary_store = {}
for strings, texts in enumerate(vocabulary):
  primary_store[texts] = strings

In [6]:
# PREVIEW OUTPUT ::

# print(vocabulary[:50])
# len(vocabulary)

In [7]:
# Splitting data into Train sets and test sets

X = [] 
y = []

for c in lines:
  xxxx = c.replace('\n','').split(' ')
  X.append(' '.join(xxxx[:-1])) # X from the corpus

  yyyy = [0 for i in range(len(vocabulary))] # Generate Y from the Vocabulary
  # yyyy[primary_store[xxxx[-1]]] = 1
  yyyy[primary_store[xxxx[-1]]] = 1
  y.append(yyyy)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
y_test = numpy.array(y_test)
y_train = numpy.array(y_train)

In [8]:
# PREVIEW OUTPUT ::

# print(X_train[:10])
# print(y_train[:10])
# print(X_test[:10])
# print(y_test[:10])

## **Embeddings!**

In [9]:
# Import the Universal Sentence Encoder's TF Hub module (Here we're making use of version 4)
# This will take a while but won't be long :)

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"  
appreciate = hub.load(module_url)

# Making it easier - Function for embedding
def embed(goodness):
  return appreciate(goodness)


In [10]:
# REVIEW OUTPUT ::

# appreciate.variables

In [11]:
# Wrapping up with the U-S-E

X_train = embed(X_train)
X_test = embed(X_test)
X_train = X_train.numpy()
X_test = X_test.numpy()

In [12]:
# PREVIEW OUTPUT ::

# print(X_train[:10])
# print(y_train[:10])
# print(X_test[:10])
# print(y_test[:10])
# print(X_train.shape, X_test.shape, y_test.shape, y_train.shape)

# **Building the model**

In [13]:
model = Sequential()
# model.add(Embedding(input_dim=len(vocabulary), output_dim=100))
model = Sequential()
# model.add(LSTM(units=100, input_shape=[512]))
model.add(Dense(512, input_shape=[512], activation = 'relu'))
model.add(Dense(units=len(vocabulary), activation = 'softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dense_1 (Dense)              (None, 2694)              1382022   
Total params: 1,644,678
Trainable params: 1,644,678
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Training the model. 

model.fit(X_train, y_train, batch_size=512, shuffle=True, epochs=20, validation_data=(X_test, y_test), callbacks=[LambdaCallback()])

Epoch 1/20
11/11 [==============================] - 1s 51ms/step - loss: 7.7687 - acc: 0.0569 - val_loss: 6.9287 - val_acc: 0.0511
Epoch 2/20
11/11 [==============================] - 0s 19ms/step - loss: 6.3200 - acc: 0.0485 - val_loss: 4.4571 - val_acc: 0.0572
Epoch 3/20
11/11 [==============================] - 0s 17ms/step - loss: 4.2940 - acc: 0.0604 - val_loss: 4.0937 - val_acc: 0.0900
Epoch 4/20
11/11 [==============================] - 0s 16ms/step - loss: 4.0394 - acc: 0.1304 - val_loss: 3.8953 - val_acc: 0.2056
Epoch 5/20
11/11 [==============================] - 0s 15ms/step - loss: 3.8269 - acc: 0.1694 - val_loss: 3.6589 - val_acc: 0.2600
Epoch 6/20
11/11 [==============================] - 0s 15ms/step - loss: 3.5807 - acc: 0.2989 - val_loss: 3.3492 - val_acc: 0.3550
Epoch 7/20
11/11 [==============================] - 0s 15ms/step - loss: 3.2411 - acc: 0.3852 - val_loss: 2.9764 - val_acc: 0.5217
Epoch 8/20
11/11 [==============================] - 0s 15ms/step - loss: 2.8520 - a

#**Unto the tests!**

In [15]:

# Create function to predict and show detailed output

def next_word(collection=[], extent=1):

  for item in collection:
    text = item
    for i in range(extent):
      prediction = model.predict(x=embed([item]).numpy())
      idx = np.argmax(prediction[-1])
      item += ' ' + vocabulary[idx]
      
      print(text + ' --> ' + item + '\nNEXT WORD: ' + item.split(' ')[-1] + '\n')

In [16]:
# Tests - please feel free to explore

single_text = ['and some other essential']

next_word(single_text)

and some other essential --> and some other essential networks
NEXT WORD: networks



In [17]:
# Testing on a collection of words

text_collection = ['deep convolutional', 'simple and effective', 'a nonconvex', 'a']

next_word(text_collection)

deep convolutional --> deep convolutional networks
NEXT WORD: networks

simple and effective --> simple and effective acceleration
NEXT WORD: acceleration

a nonconvex --> a nonconvex dataset
NEXT WORD: dataset

a --> a improvement
NEXT WORD: improvement



## **For the record**

The Dataset is based on a Tensorflow tutorial from Stanford, so all predicted words will be based on Deep learning and Machine learning _common terms_.



In [18]:
# Storing data

vocabulary = numpy.array(vocabulary)
numpy.save('./vocabulary.npy', vocabulary)
model.save('./NWP-USE')

INFO:tensorflow:Assets written to: ./NWP-USE/assets


INFO:tensorflow:Assets written to: ./NWP-USE/assets


In [19]:
##                                                                  END OF NOTEBOOK